In [1]:
!pip install ta
#데이터셋을 만들어 주는 함수
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.layers import LSTM,Dropout,GRU
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

df = pd.read_csv("./[경기관련소비재]현대차.csv")
df_columns = ['Date','Open', 'High', 'Low', 'Close', 'Volume']
df.columns = df_columns
df = df.sort_values('Date')
df['Date']= df['Date'].astype('str')
from datetime import datetime
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace=True)

ma = [5,20,60,120]
for days in ma:
    df['ma_'+str(days)] = df['Close'].rolling(window = days).mean()
    df['vma_'+str(days)] = df['Volume'].rolling(window = days).mean()

df.dropna(inplace=True)


import ta

H, L, C, V = df['High'], df['Low'], df['Close'], df['Volume']

# df['bol_high'] = ta.volatility.bollinger_hband(C)
# df['bol_low']  = ta.volatility.bollinger_lband(C)
df['MFI'] = ta.volume.money_flow_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['ADI'] = ta.volume.acc_dist_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['OBV'] = ta.volume.on_balance_volume(close=C, volume=V, fillna=True)
df['CMF'] = ta.volume.chaikin_money_flow(
    high=H, low=L, close=C, volume=V, fillna=True)

df['FI'] = ta.volume.force_index(close=C, volume=V, fillna=True)
df['EOM, EMV'] = ta.volume.ease_of_movement(
    high=H, low=L, volume=V, fillna=True)

df['VPT'] = ta.volume.volume_price_trend(close=C, volume=V, fillna=True)
df['NVI'] = ta.volume.negative_volume_index(close=C, volume=V, fillna=True)
df['VMAP'] = ta.volume.volume_weighted_average_price(
    high=H, low=L, close=C, volume=V, fillna=True)

# Volatility
df['ATR'] = ta.volatility.average_true_range(
    high=H, low=L, close=C, fillna=True)
df['BHB'] = ta.volatility.bollinger_hband(close=C, fillna=True)
df['BLB'] = ta.volatility.bollinger_lband(close=C, fillna=True)
df['KCH'] = ta.volatility.keltner_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['KCL'] = ta.volatility.keltner_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['KCM'] = ta.volatility.keltner_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['DCH'] = ta.volatility.donchian_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['DCL'] = ta.volatility.donchian_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['DCM'] = ta.volatility.donchian_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['UI'] = ta.volatility.ulcer_index(close=C, fillna=True)
# Trend
df['SMA'] = ta.trend.sma_indicator(close=C, fillna=True)
df['EMA'] = ta.trend.ema_indicator(close=C, fillna=True)
df['WMA'] = ta.trend.wma_indicator(close=C, fillna=True)
df['MACD'] = ta.trend.macd(close=C, fillna=True)
df['ADX'] = ta.trend.adx(high=H, low=L, close=C, fillna=True)
df['-VI'] = ta.trend.vortex_indicator_neg(
    high=H, low=L, close=C, fillna=True)
df['+VI'] = ta.trend.vortex_indicator_pos(
    high=H, low=L, close=C, fillna=True)
df['TRIX'] = ta.trend.trix(close=C, fillna=True)
df['MI'] = ta.trend.mass_index(high=H, low=L, fillna=True)
df['CCI'] = ta.trend.cci(high=H, low=L, close=C, fillna=True)
df['DPO'] = ta.trend.dpo(close=C, fillna=True)
df['KST'] = ta.trend.kst(close=C, fillna=True)
df['Ichimoku'] = ta.trend.ichimoku_a(high=H, low=L, fillna=True)
df['Parabolic SAR'] = ta.trend.psar_down(
    high=H, low=L, close=C, fillna=True)
df['STC'] = ta.trend.stc(close=C, fillna=True)
# Momentum
df['RSI'] = ta.momentum.rsi(close=C, fillna=True)
df['SRSI'] = ta.momentum.stochrsi(close=C, fillna=True)
df['TSI'] = ta.momentum.tsi(close=C, fillna=True)
df['UO'] = ta.momentum.ultimate_oscillator(
    high=H, low=L, close=C, fillna=True)
df['SR'] = ta.momentum.stoch(close=C, high=H, low=L, fillna=True)
df['WR'] = ta.momentum.williams_r(high=H, low=L, close=C, fillna=True)
df['AO'] = ta.momentum.awesome_oscillator(high=H, low=L, fillna=True)
df['KAMA'] = ta.momentum.kama(close=C, fillna=True)
df['ROC'] = ta.momentum.roc(close=C, fillna=True)
df['PPO'] = ta.momentum.ppo(close=C, fillna=True)
df['PVO'] = ta.momentum.pvo(volume=V, fillna=True)


from sklearn.preprocessing import MinMaxScaler

# 피처값 스케일링
scaler = MinMaxScaler()

scaled_df = scaler.fit_transform(df.drop(['Close'], axis=1))
scaled_df = pd.DataFrame(scaled_df, columns = df.drop(['Close'], axis=1).columns)

scaler1 = MinMaxScaler()

temp = scaler1.fit_transform(df['Close'].values.reshape(-1,1))

scaled_df.insert(3, 'Close',temp)
scaled_df

df = scaled_df



import numpy as np

def make_dataset(feature, label, window_size = 20):
  
  feature_list = []
  label_list = []
  
  for i in range(len(feature) - window_size) :
    feature_list.append(feature.iloc[i:i+window_size])
    label_list.append(label.iloc[i+window_size])
  
  return np.array(feature_list), np.array(label_list)

WINDOW_SIZE=20

result = []
predict = []


####최근 20개의 데이터만
pred_feature = df[-WINDOW_SIZE :]
pred_feature = pred_feature.to_numpy()

pred_feature = pred_feature.reshape((1,20,-1))
# pred_feature.shape

import random as rn
import tensorflow as tf


for i in range(1, len(df)//1000+2):

        # 데이터 불러오기
        df_copy = df
     
        # 원하는 크기로 데이터 자르기
        df_copy = df_copy[-i*1000:]
        # 피처값, 타켓 스케일링
        
        TEST_SIZE = int(len(df_copy) * 0.7)
        
        train = df_copy[:TEST_SIZE]
        test = df_copy[TEST_SIZE:]

##200일의 데이터를 얼마나 잘 예측하느냐
        
        # feature_cols = df.drop('Close', axis =1).columns
        feature_cols = df.columns
        label_cols = ['Close']

        train_feature = train[feature_cols]
        train_label = train[label_cols]

        train_feature, train_label = make_dataset(train_feature, train_label,20)

        from sklearn.model_selection import train_test_split
        x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2, shuffle = False)
        
        
        test_feature = test[feature_cols]
        test_label = test[label_cols]
        test_feature , test_label = make_dataset(test_feature, test_label,20)
        
        
        from keras.layers.normalization import batch_normalization

        model = Sequential()
        model.add(GRU(64, 
                      input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                      return_sequences = False,
                      activation='relu'))
        model.add(Dense(1))
      
        
        model.compile(loss='mean_squared_error', optimizer='adam')

        early_stop = EarlyStopping(monitor='val_loss', patience=10)

        import os
        from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

        model_path = 'model'
        filename = os.path.join(model_path, 'tmp_checkpoint.h5')
        checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

        history = model.fit(x_train, y_train, 
                                            epochs=100, 
                                            batch_size=16,
                                            validation_data=(x_valid, y_valid), 
                                            callbacks=[early_stop, checkpoint])


        pred = model.predict(test_feature)
        # rescaleing 작업
        rescaled_y_true = scaler1.inverse_transform(np.array(test_label).reshape(-1, 1))
        rescaled_pred = scaler1.inverse_transform(np.array(pred).reshape(-1,1))
        
        
        # 평가지표(RMSE) 계산
        RMSE = np.sqrt(mean_squared_error(rescaled_y_true, rescaled_pred))
        result.append(RMSE)
        print(f"{i * 1000}길이의 데이터 적용 완료\n 길이: {i * 1000}, RMSE:{RMSE}")
        print(result)
        
        

        new_pred = model.predict(pred_feature)
        rescaled_newpred = scaler1.inverse_transform(np.array(new_pred).reshape(-1,1))
        predict.append(rescaled_newpred)



C:\Users\gram\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\gram\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Epoch 1/100
30/34 [=========================>....] - ETA: 0s - loss: 0.0675
Epoch 1: val_loss improved from inf to 0.00623, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 3s 29ms/step - loss: 0.0599 - val_loss: 0.0062
Epoch 2/100
29/34 [========================>.....] - ETA: 0s - loss: 0.0024
Epoch 2: val_loss did not improve from 0.00623
34/34 [==============================] - 0s 10ms/step - loss: 0.0022 - val_loss: 0.0182
Epoch 3/100
29/34 [========================>.....] - ETA: 0s - loss: 8.4737e-04
Epoch 3: val_loss did not improve from 0.00623
34/34 [==============================] - 0s 12ms/step - loss: 8.0975e-04 - val_loss: 0.0112
Epoch 4/100
29/34 [========================>.....] - ETA: 0s - loss: 6.4577e-04
Epoch 4: val_loss did not improve from 0.00623
34/34 [==============================] - 0s 10ms/step - loss: 6.3402e-04 - val_loss: 0.0095
Epoch 5/100
28/34 [=======================>......] - ETA: 0s - loss: 5.5204e-04
Epoch 5: val_loss d

28/34 [=======================>......] - ETA: 0s - loss: 1.5561e-04
Epoch 36: val_loss did not improve from 0.00161
34/34 [==============================] - 0s 11ms/step - loss: 1.4765e-04 - val_loss: 0.0021
Epoch 37/100
30/34 [=========================>....] - ETA: 0s - loss: 1.4451e-04
Epoch 37: val_loss improved from 0.00161 to 0.00160, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 0s 12ms/step - loss: 1.5250e-04 - val_loss: 0.0016
Epoch 38/100
30/34 [=========================>....] - ETA: 0s - loss: 1.6226e-04
Epoch 38: val_loss did not improve from 0.00160
34/34 [==============================] - 0s 9ms/step - loss: 1.5830e-04 - val_loss: 0.0020
Epoch 39/100
33/34 [============================>.] - ETA: 0s - loss: 1.4603e-04
Epoch 39: val_loss did not improve from 0.00160
34/34 [==============================] - 0s 9ms/step - loss: 1.4568e-04 - val_loss: 0.0019
Epoch 40/100
29/34 [========================>.....] - ETA: 0s - loss: 1.7927e-04
Epoch

69/69 [==============================] - 1s 8ms/step - loss: 1.9288e-04 - val_loss: 2.5517e-04
Epoch 25/100
65/69 [===========================>..] - ETA: 0s - loss: 2.1256e-04
Epoch 25: val_loss improved from 0.00011 to 0.00009, saving model to model\tmp_checkpoint.h5
69/69 [==============================] - 1s 10ms/step - loss: 2.1661e-04 - val_loss: 8.8251e-05
Epoch 26/100
61/69 [=========================>....] - ETA: 0s - loss: 2.2566e-04
Epoch 26: val_loss did not improve from 0.00009
69/69 [==============================] - 1s 8ms/step - loss: 2.1720e-04 - val_loss: 1.4131e-04
Epoch 27/100
67/69 [============================>.] - ETA: 0s - loss: 1.7209e-04
Epoch 27: val_loss did not improve from 0.00009
69/69 [==============================] - 1s 8ms/step - loss: 1.7024e-04 - val_loss: 1.7582e-04
Epoch 28/100
62/69 [=========================>....] - ETA: 0s - loss: 2.0742e-04
Epoch 28: val_loss did not improve from 0.00009
69/69 [==============================] - 1s 7ms/step - los

Epoch 7/100
104/104 [==============================] - ETA: 0s - loss: 3.7438e-04
Epoch 7: val_loss improved from 0.00029 to 0.00027, saving model to model\tmp_checkpoint.h5
104/104 [==============================] - 1s 8ms/step - loss: 3.7438e-04 - val_loss: 2.6996e-04
Epoch 8/100
104/104 [==============================] - ETA: 0s - loss: 3.8436e-04
Epoch 8: val_loss improved from 0.00027 to 0.00026, saving model to model\tmp_checkpoint.h5
104/104 [==============================] - 1s 9ms/step - loss: 3.8436e-04 - val_loss: 2.6137e-04
Epoch 9/100
100/104 [===========================>..] - ETA: 0s - loss: 3.7114e-04
Epoch 9: val_loss improved from 0.00026 to 0.00026, saving model to model\tmp_checkpoint.h5
104/104 [==============================] - 1s 9ms/step - loss: 3.6474e-04 - val_loss: 2.5651e-04
Epoch 10/100
102/104 [============================>.] - ETA: 0s - loss: 3.6427e-04
Epoch 10: val_loss improved from 0.00026 to 0.00025, saving model to model\tmp_checkpoint.h5
104/104 [==

...

206/209 [============================>.] - ETA: 0s - loss: 1.0710e-04
Epoch 23: val_loss did not improve from 0.00030
209/209 [==============================] - 2s 9ms/step - loss: 1.0667e-04 - val_loss: 3.4954e-04
Epoch 24/100
206/209 [============================>.] - ETA: 0s - loss: 9.8233e-05
Epoch 24: val_loss did not improve from 0.00030
209/209 [==============================] - 2s 8ms/step - loss: 9.8592e-05 - val_loss: 6.7036e-04
Epoch 25/100
205/209 [============================>.] - ETA: 0s - loss: 1.0990e-04
Epoch 25: val_loss improved from 0.00030 to 0.00028, saving model to model\tmp_checkpoint.h5
209/209 [==============================] - 2s 9ms/step - loss: 1.0927e-04 - val_loss: 2.8148e-04
Epoch 26/100
203/209 [============================>.] - ETA: 0s - loss: 1.1270e-04
Epoch 26: val_loss improved from 0.00028 to 0.00028, saving model to model\tmp_checkpoint.h5
209/209 [==============================] - 2s 8ms/step - loss: 1.1079e-04 - val_loss: 2.7967e-04
Epoch 27/10

Epoch 14/100
243/244 [============================>.] - ETA: 0s - loss: 4.6378e-05
Epoch 14: val_loss did not improve from 0.00496
244/244 [==============================] - 2s 8ms/step - loss: 4.6358e-05 - val_loss: 0.0076
Epoch 15/100
243/244 [============================>.] - ETA: 0s - loss: 5.1800e-05
Epoch 15: val_loss did not improve from 0.00496
244/244 [==============================] - 2s 9ms/step - loss: 5.1842e-05 - val_loss: 0.0050
Epoch 16/100
244/244 [==============================] - ETA: 0s - loss: 3.8865e-05
Epoch 16: val_loss did not improve from 0.00496
244/244 [==============================] - 2s 9ms/step - loss: 3.8865e-05 - val_loss: 0.0072
Epoch 17/100
239/244 [============================>.] - ETA: 0s - loss: 4.8712e-05
Epoch 17: val_loss did not improve from 0.00496
244/244 [==============================] - 3s 11ms/step - loss: 4.8367e-05 - val_loss: 0.0087
Epoch 18/100
243/244 [============================>.] - ETA: 0s - loss: 4.5217e-05
Epoch 18: val_loss d

276/279 [============================>.] - ETA: 0s - loss: 3.0680e-05
Epoch 24: val_loss improved from 0.00732 to 0.00635, saving model to model\tmp_checkpoint.h5
279/279 [==============================] - 2s 7ms/step - loss: 3.0691e-05 - val_loss: 0.0064
Epoch 25/100
273/279 [============================>.] - ETA: 0s - loss: 1.9755e-05
Epoch 25: val_loss did not improve from 0.00635
279/279 [==============================] - 2s 7ms/step - loss: 1.9622e-05 - val_loss: 0.0065
Epoch 26/100
275/279 [============================>.] - ETA: 0s - loss: 2.3239e-05
Epoch 26: val_loss did not improve from 0.00635
279/279 [==============================] - 2s 7ms/step - loss: 2.3404e-05 - val_loss: 0.0082
Epoch 27/100
279/279 [==============================] - ETA: 0s - loss: 2.8228e-05
Epoch 27: val_loss did not improve from 0.00635
279/279 [==============================] - 2s 7ms/step - loss: 2.8228e-05 - val_loss: 0.0072
Epoch 28/100
279/279 [==============================] - ETA: 0s - loss: 

310/314 [============================>.] - ETA: 0s - loss: 3.4005e-05
Epoch 11: val_loss did not improve from 0.00125
314/314 [==============================] - 2s 7ms/step - loss: 3.3709e-05 - val_loss: 0.0016
Epoch 12/100
313/314 [============================>.] - ETA: 0s - loss: 2.1303e-05
Epoch 12: val_loss did not improve from 0.00125
314/314 [==============================] - 2s 7ms/step - loss: 2.1366e-05 - val_loss: 0.0020
Epoch 13/100
311/314 [============================>.] - ETA: 0s - loss: 2.4501e-05
Epoch 13: val_loss improved from 0.00125 to 0.00107, saving model to model\tmp_checkpoint.h5
314/314 [==============================] - 2s 7ms/step - loss: 2.4580e-05 - val_loss: 0.0011
Epoch 14/100
314/314 [==============================] - ETA: 0s - loss: 2.3088e-05
Epoch 14: val_loss did not improve from 0.00107
314/314 [==============================] - 2s 7ms/step - loss: 2.3088e-05 - val_loss: 0.0013
Epoch 15/100
313/314 [============================>.] - ETA: 0s - loss: 

340/341 [============================>.] - ETA: 0s - loss: 2.2605e-05
Epoch 9: val_loss did not improve from 0.00008
341/341 [==============================] - 4s 12ms/step - loss: 2.2614e-05 - val_loss: 2.2013e-04
Epoch 10/100
337/341 [============================>.] - ETA: 0s - loss: 2.0305e-05
Epoch 10: val_loss did not improve from 0.00008
341/341 [==============================] - 4s 12ms/step - loss: 2.0173e-05 - val_loss: 1.6248e-04
Epoch 11/100
335/341 [============================>.] - ETA: 0s - loss: 1.8922e-05
Epoch 11: val_loss improved from 0.00008 to 0.00008, saving model to model\tmp_checkpoint.h5
341/341 [==============================] - 3s 10ms/step - loss: 1.8834e-05 - val_loss: 7.9847e-05
Epoch 12/100
341/341 [==============================] - ETA: 0s - loss: 2.8793e-05
Epoch 12: val_loss did not improve from 0.00008
341/341 [==============================] - 3s 9ms/step - loss: 2.8793e-05 - val_loss: 1.1473e-04
Epoch 13/100
338/341 [============================>.] 

In [2]:
print(predict)

[array([[184194.95]], dtype=float32), array([[186150.8]], dtype=float32), array([[191926.77]], dtype=float32), array([[188072.34]], dtype=float32), array([[194991.38]], dtype=float32), array([[189213.52]], dtype=float32), array([[185668.84]], dtype=float32), array([[166508.05]], dtype=float32), array([[165920.94]], dtype=float32), array([[183705.25]], dtype=float32)]
